In [ ]:
import numpy as np
import PIL.Image
import tensorflow as tf
import pathlib
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Dense

In [ ]:
data_dir = pathlib.Path('./Datasets/lung/').with_suffix('')
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
train_ds:tf.data.Dataset = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="training",
  label_mode="binary",
  seed=123,
  image_size=(512, 512),
  batch_size=32) # type: ignore

In [ ]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="validation",
  label_mode="binary",
  seed=123,
  image_size=(512, 512),
  batch_size=32)

In [ ]:
class_names_ = train_ds.class_names # type: ignore
print(class_names_)

In [ ]:
model=Sequential()
model.add(Conv2D(filters= 8,kernel_size=(4,4),input_shape=(512,512,3),padding='same'))
model.add(Conv2D(filters= 16,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(filters= 32,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(units=16,activation='relu'))
model.add(Dense(units=8,activation='relu'))
model.add(Dense(1))
model.summary()

In [ ]:
import keras.backend as K
def recall(y_true, y_pred):
    y_true = K.ones_like(y_true)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    all_positives = K.sum(K.round(K.clip(y_true, 0, 1)))

    recall = true_positives / (all_positives + K.epsilon())
    return recall

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

In [ ]:
model.compile(
  optimizer='adam',
  loss=keras.losses.BinaryCrossentropy(from_logits=True),
  metrics=['accuracy',specificity,recall])

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
es=EarlyStopping(monitor="val_accuracy",min_delta=0.01,patience=3,verbose=1,mode='auto')
mc=ModelCheckpoint(monitor="val_accuracy",filepath="./bestmodel.h5",verbose=1,save_best_only=True,mode='auto')
cd=[es,mc]

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10,
  use_multiprocessing=True,
  callbacks=cd
)

In [ ]:
from keras.preprocessing.image import load_img,img_to_array
path="./Datasets/lung/Normal cases/Normal case (5).jpg"
img=load_img(path,target_size=(512,512))

In [ ]:
arr=img_to_array(img)
arr=np.expand_dims(arr,axis=0)
print(arr.shape)
pred=model.predict(arr)
print(pred)
pred_1=list(pred[0])
print(pred_1)
ans=list(class_names_)
print(ans)
print(ans[pred_1.index(max(pred_1))])